# Building Portfolios from Synthetic Assets

In [ ]:
################
# Import dependencies
################
import numpy as np
import matplotlib.pyplot as plt
#import correlations as corr
import distcorr as corr

In [ ]:
##################
# Define global variables
##################
L = 20 #lookback window
rebalance = 5 #rebalancing frequency

## Lead-Lag Metrics

In [ ]:
def ccf(x, y, lags=None):
    """
    Cross-correlation function
    assume x,y are already loaded in for correct time frame
    """
    if lags is None:
        lags = np.arange(-len(x) + 1, len(y))
    
    ccf_values = []
    for lag in lags:
        if lag < 0: #this refers to x leading y
            cross_corr = corr.distance_correlation(x[:lag], y[-lag:]) 
            ccf_values.append(cross_corr)
        else: #this refers to y leading x
            cross_corr = corr.distance_correlation(x[lag:], y[:-lag])
            ccf_values.append(cross_corr)
    
    return np.array(ccf_values)

def ccf_auc(x, y, lag):
    """
    Area under the cross-correlation function
    """
    lags = list(range(1, lag + 1))
    ccf_values_xy = ccf(x, y, lags)
    ccf_values_yx = ccf(y, x, lags)
    I_xy, I_yx = 0 #corresponds to correlation of x leading y and y leading x
    for i in lags:
        I_xy += abs(ccf_values_xy[i - 1])
        I_yx += abs(ccf_values_yx[i - 1])
    max_I = max(I_xy, I_yx)
    auc = np.sign(I_xy - I_yx) * (max_I / (I_xy + I_yx))
    return auc



## GlobalRank